In [121]:
import pandas as pd
import numpy as np
from datetime import datetime, date, time
import os
import matplotlib.pyplot as plt
#from prophet import Prophet
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

In [ ]:
local_download_path = '/Users/heitung/Desktop/Final Project'

all_files = os.listdir(local_download_path)
csv_files = [file for file in all_files if file.endswith('.csv')]

dfs = []
for file in csv_files:
    file_path = local_download_path+"/"+file
    print(file_path)
    df =pd.read_csv(file,parse_dates=['Transaction Date'])
    if not any(df.equals(existing_df) for existing_df in dfs):
        dfs.append(df)

all_data = pd.concat(dfs, axis=0, ignore_index=True)


In [257]:
all_data.replace("-", np.nan, inplace=True)
all_data['Transaction Date'].dtypes

dtype('O')

In [258]:
clean_df = all_data.dropna(subset=['Transaction Date', 'Staff No.','Staff Name', 'Gender','Site',])
clean_df

,REF#,Category,Transaction Date,Staff No.,Staff Name,Gender,Site,Ranking,Remarks,Remark/Size,...,B004-褲/裙\nPants / Skirt-MTR 新 男灰色西褲-46/7XL,B004-褲/裙\nPants / Skirt-MTR 新 男灰色西褲-48/8XL,B004-褲/裙\nPants / Skirt-MTR 新 男灰色西褲-56/12XL,B005-褲/裙\nPants / Skirt-MTR 新 女灰色西裙-42/5XL,B005-褲/裙\nPants / Skirt-MTR 新 女灰色西裙-44/6XL,A052-西裝上衣\nSuit\nTop-N393-8 良景女裝灰色西裝褸 (MYLKC006)-3XL,A053-西裝上衣\nSuit\nTop-N393-8 良景女裝白色恤衫 (MYLKC007)-16/L,A053-西裝上衣\nSuit\nTop-N393-8 良景女裝白色恤衫 (MYLKC007)-18/2XL,B004-褲/裙\nPants / Skirt-MTR 新 男灰色西褲-50/10XL,B005-褲/裙\nPants / Skirt-MTR 新 女灰色西裙-46/7XL
0,220901-0001,派發 (全新),1/9/2022,A59110,林梓凱,M,PT,AUX,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,220901-0002,派發 (全新),1/9/2022,A71206,余朝勝,M,H364,SG,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,220901-0003,派發 (全新),1/9/2022,069145,吳永雄,M,NT637A,SG,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,220901-0006,派發 (全新),1/9/2022,039281,林杏權,M,N367,SG,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,220901-0007,派發 (重用),1/9/2022,039281,林杏權,M,N367,SG,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11449,221230-0033,派發 (重用),30/12/2022,074445,戴智昌,M,H392,SG,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11450,221202-0038,派發 (重用),2/12/2022,A73192,李燕妮娜,F,N373-6,SG,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11451,221202-0039,派發 (重用),2/12/2022,063624,李佩清,M,H313,SO,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11452,221201-0044,派發 (重用),1/12/2022,A65384W,林春霞,F,REL,SG,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [259]:
grouped_df = clean_df.groupby(["Transaction Date", "Staff No.", "Staff Name", "Gender", "Site", "Ranking"]).sum().reset_index()
grouped_df =grouped_df.drop(['REF#','Remarks','Remark/Size','Category'], axis=1)
grouped_df


,Transaction Date,Staff No.,Staff Name,Gender,Site,Ranking,A001-軍裝上衣\nMilitary Top-軍灰紫恤衫(軍)-14,A001-軍裝上衣\nMilitary Top-軍灰紫恤衫(軍)-15,A001-軍裝上衣\nMilitary Top-軍灰紫恤衫(軍)-16,A001-軍裝上衣\nMilitary Top-軍灰紫恤衫(軍)-17,...,B004-褲/裙\nPants / Skirt-MTR 新 男灰色西褲-46/7XL,B004-褲/裙\nPants / Skirt-MTR 新 男灰色西褲-48/8XL,B004-褲/裙\nPants / Skirt-MTR 新 男灰色西褲-56/12XL,B005-褲/裙\nPants / Skirt-MTR 新 女灰色西裙-42/5XL,B005-褲/裙\nPants / Skirt-MTR 新 女灰色西裙-44/6XL,A052-西裝上衣\nSuit\nTop-N393-8 良景女裝灰色西裝褸 (MYLKC006)-3XL,A053-西裝上衣\nSuit\nTop-N393-8 良景女裝白色恤衫 (MYLKC007)-16/L,A053-西裝上衣\nSuit\nTop-N393-8 良景女裝白色恤衫 (MYLKC007)-18/2XL,B004-褲/裙\nPants / Skirt-MTR 新 男灰色西褲-50/10XL,B005-褲/裙\nPants / Skirt-MTR 新 女灰色西裙-46/7XL
0,0/1/1900,A66629F,景月霞,F,REL,SG,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1/11/2022,059311,陳鳳芝,F,H313,SO,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1/11/2022,061661,鍾志財,M,K311,SSG,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1/11/2022,065507,周笑強,M,H231,C/INSP,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1/11/2022,072757,鄧倩雯,F,KT794A,SG,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6049,9/9/2022,072860,顏協成,M,KT794A,SG,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6050,9/9/2022,A67671F,黃錦輝,M,REL,SG,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6051,9/9/2022,A69573W,陳立志,M,REL,SG,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6052,9/9/2022,A71008,洪麗環,F,REL,SG,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [260]:
melted_detail_df = pd.melt(grouped_df,
                           id_vars=["Transaction Date", "Staff No.", "Staff Name", "Gender", "Site", "Ranking"],
                           value_name="Out", var_name="Item Name")
melted_detail_df = melted_detail_df[melted_detail_df['Out'] != 0]

cleaned_df = melted_detail_df.groupby(["Transaction Date", "Staff No.", "Staff Name", "Gender", "Site", "Ranking", "Item Name"])["Out"].sum().reset_index()

In [261]:
def is_float(value):
    try:
        float(value)
        return True
    except ValueError:
        return False

a_float = cleaned_df['Out'].apply(is_float)

cleaned_df = cleaned_df[a_float]
cleaned_df['Out'] =cleaned_df['Out'].astype(float)
cleaned_df

,Transaction Date,Staff No.,Staff Name,Gender,Site,Ranking,Item Name,Out
0,0/1/1900,A66629F,景月霞,F,REL,SG,A043-冷衫\nSWEATER-V領羊毛背心(西)-S,1.0
1,1/11/2022,059311,陳鳳芝,F,H313,SO,A003-軍裝上衣\nMilitary Top-MTR短灰恤衫(軍)-16,2.0
2,1/11/2022,059311,陳鳳芝,F,H313,SO,A029-風褸\nJacket-MTR風褸(軍)-M,1.0
3,1/11/2022,059311,陳鳳芝,F,H313,SO,C021-肩章\nEpaulettes -MTR肩章(三柴)(軍)-三柴,1.0
4,1/11/2022,059311,陳鳳芝,F,H313,SO,C064-腰物\nBelt and Accessory-MTR腰包(軍)--,1.0
...,...,...,...,...,...,...,...,...
38704,9/9/2022,A71008,洪麗環,F,REL,SG,C059-腰物\nBelt and Accessory-軍腰帶(軍)--,1.0
38705,9/9/2022,A71008,洪麗環,F,REL,SG,C063-腰物\nBelt and Accessory-Certis腰包(軍)--,1.0
38706,9/9/2022,A71045,張嘉恬,F,REL,SG,A007-西裝上衣\nSuit\nTop-長白恤衫(西)-14,1.0
38707,9/9/2022,A71045,張嘉恬,F,REL,SG,A022-西裝褸\nSuit-女西裝褸(西)-M,1.0


In [262]:
split_columns = cleaned_df['Item Name'].str.split('-', expand=True)
split_columns.columns = ["Code", "Category", "Item","size", "test", "test2","test3"]
split_columns["Category"] = split_columns["Category"].str.replace('\n', ' ')
split_columns['Item_name'] = split_columns['Item'].astype(str) + " " + split_columns['size'].astype(str)
split_columns = split_columns.drop(["Item",	"size",	"test","test2",	"test3"], axis=1)

split_columns

,Code,Category,Item_name
0,A043,冷衫 SWEATER,V領羊毛背心(西) S
1,A003,軍裝上衣 Military Top,MTR短灰恤衫(軍) 16
2,A029,風褸 Jacket,MTR風褸(軍) M
3,C021,肩章 Epaulettes,MTR肩章(三柴)(軍) 三柴
4,C064,腰物 Belt and Accessory,MTR腰包(軍)
...,...,...,...
38704,C059,腰物 Belt and Accessory,軍腰帶(軍)
38705,C063,腰物 Belt and Accessory,Certis腰包(軍)
38706,A007,西裝上衣 Suit Top,長白恤衫(西) 14
38707,A022,西裝褸 Suit,女西裝褸(西) M


In [263]:
final_df = pd.concat(([split_columns, (cleaned_df.drop(['Item Name', 'Staff Name'],axis =1))]), axis=1)
final_df

,Code,Category,Item_name,Transaction Date,Staff No.,Gender,Site,Ranking,Out
0,A043,冷衫 SWEATER,V領羊毛背心(西) S,0/1/1900,A66629F,F,REL,SG,1.0
1,A003,軍裝上衣 Military Top,MTR短灰恤衫(軍) 16,1/11/2022,059311,F,H313,SO,2.0
2,A029,風褸 Jacket,MTR風褸(軍) M,1/11/2022,059311,F,H313,SO,1.0
3,C021,肩章 Epaulettes,MTR肩章(三柴)(軍) 三柴,1/11/2022,059311,F,H313,SO,1.0
4,C064,腰物 Belt and Accessory,MTR腰包(軍),1/11/2022,059311,F,H313,SO,1.0
...,...,...,...,...,...,...,...,...,...
38704,C059,腰物 Belt and Accessory,軍腰帶(軍),9/9/2022,A71008,F,REL,SG,1.0
38705,C063,腰物 Belt and Accessory,Certis腰包(軍),9/9/2022,A71008,F,REL,SG,1.0
38706,A007,西裝上衣 Suit Top,長白恤衫(西) 14,9/9/2022,A71045,F,REL,SG,1.0
38707,A022,西裝褸 Suit,女西裝褸(西) M,9/9/2022,A71045,F,REL,SG,1.0


In [264]:
extra_date = final_df[final_df['Transaction Date']=='0/1/1900'].index
final_df.drop(extra_date, inplace =True)

In [265]:
final_df

,Code,Category,Item_name,Transaction Date,Staff No.,Gender,Site,Ranking,Out
1,A003,軍裝上衣 Military Top,MTR短灰恤衫(軍) 16,1/11/2022,059311,F,H313,SO,2.0
2,A029,風褸 Jacket,MTR風褸(軍) M,1/11/2022,059311,F,H313,SO,1.0
3,C021,肩章 Epaulettes,MTR肩章(三柴)(軍) 三柴,1/11/2022,059311,F,H313,SO,1.0
4,C064,腰物 Belt and Accessory,MTR腰包(軍),1/11/2022,059311,F,H313,SO,1.0
5,C066,腰物 Belt and Accessory,水袋(軍),1/11/2022,059311,F,H313,SO,1.0
...,...,...,...,...,...,...,...,...,...
38704,C059,腰物 Belt and Accessory,軍腰帶(軍),9/9/2022,A71008,F,REL,SG,1.0
38705,C063,腰物 Belt and Accessory,Certis腰包(軍),9/9/2022,A71008,F,REL,SG,1.0
38706,A007,西裝上衣 Suit Top,長白恤衫(西) 14,9/9/2022,A71045,F,REL,SG,1.0
38707,A022,西裝褸 Suit,女西裝褸(西) M,9/9/2022,A71045,F,REL,SG,1.0


Machine Learning Part

In [266]:
ML_df = final_df.copy()

In [267]:
ML_df['Transaction Date'] = pd.to_datetime(ML_df['Transaction Date'], format='%d/%m/%Y', errors='coerce')

In [268]:
ML_df['Month'] = ML_df['Transaction Date'].dt.strftime('%Y-%m')
ML_df

,Code,Category,Item_name,Transaction Date,Staff No.,Gender,Site,Ranking,Out,Month
1,A003,軍裝上衣 Military Top,MTR短灰恤衫(軍) 16,2022-11-01,059311,F,H313,SO,2.0,2022-11
2,A029,風褸 Jacket,MTR風褸(軍) M,2022-11-01,059311,F,H313,SO,1.0,2022-11
3,C021,肩章 Epaulettes,MTR肩章(三柴)(軍) 三柴,2022-11-01,059311,F,H313,SO,1.0,2022-11
4,C064,腰物 Belt and Accessory,MTR腰包(軍),2022-11-01,059311,F,H313,SO,1.0,2022-11
5,C066,腰物 Belt and Accessory,水袋(軍),2022-11-01,059311,F,H313,SO,1.0,2022-11
...,...,...,...,...,...,...,...,...,...,...
38704,C059,腰物 Belt and Accessory,軍腰帶(軍),2022-09-09,A71008,F,REL,SG,1.0,2022-09
38705,C063,腰物 Belt and Accessory,Certis腰包(軍),2022-09-09,A71008,F,REL,SG,1.0,2022-09
38706,A007,西裝上衣 Suit Top,長白恤衫(西) 14,2022-09-09,A71045,F,REL,SG,1.0,2022-09
38707,A022,西裝褸 Suit,女西裝褸(西) M,2022-09-09,A71045,F,REL,SG,1.0,2022-09


In [276]:
monthly_sales = ML_df.groupby(['Month', 'Code', 'Item_name'])['Out'].sum().reset_index()
# monthly_sales['Month'] = monthly_sales['Month'].dt.to_timestamp()

In [ ]:
items = []
    
for item in ML_df['Item_name'].unique():
    items.append(item)
       
X = [items]
y = ML_df['Out']
        
model = LinearRegression()
model.fit(X, y)
        
        # next_month_index = np.array([[item_data['Month_Index'].max() + 1]])
next_month_sales_forecast = model.predict(items.count)
        
        # forecasts.append((item, next_month_sales_forecast[0]))

In [277]:

def forecast_sales(df):
    
    forecasts = []
    
    for item in df['Item_name'].unique():
        item_data = df[df['Item_name'] == item].copy()
        
        if len(item_data) < 2:
            continue
        
        item_data['Month_Index'] = range(1, len(item_data) + 1)
        
        X = item_data[['Month_Index']]
        y = item_data['Out']
        
        model = LinearRegression()
        model.fit(X, y)
        
        next_month_index = np.array([[item_data['Month_Index'].max() + 1]])
        next_month_sales_forecast = model.predict(next_month_index)
        
        forecasts.append((item, next_month_sales_forecast[0]))
    
    forecast_df = pd.DataFrame(forecasts, columns=['Item_name', 'Forecasted_Sales'])
    
    return forecast_df




In [ ]:
sales_forecasts = forecast_sales(monthly_sales)
sales_forecasts.head()
sales_forecasts['Forecasted_Sales'] = np.round(sales_forecasts['Forecasted_Sales'], 0).astype(int)

In [297]:
monthly_sales_by_month = ML_df.groupby(['Month', 'Item_name','Category'])['Out'].sum().reset_index()
monthly_sales= monthly_sales_by_month.pivot(index=['Item_name','Category'], columns='Month', values='Out').fillna(0).reset_index()
monthly_sales

Month,Item_name,Category,2022-08,2022-09,2022-10,2022-11,2022-12,2023-01,2023-02,2023-03,2023-04,2023-05,2023-06,2023-07,2023-08
0,Certis反光衣(軍) 2XL,反光衣 Reflective Vest,0.0,2.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,2.0
1,Certis反光衣(軍) 3XL,反光衣 Reflective Vest,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
2,Certis反光衣(軍) L,反光衣 Reflective Vest,5.0,4.0,2.0,4.0,5.0,5.0,1.0,1.0,3.0,10.0,4.0,3.0,7.0
3,Certis反光衣(軍) M,反光衣 Reflective Vest,5.0,7.0,2.0,1.0,6.0,3.0,2.0,2.0,3.0,7.0,4.0,1.0,3.0
4,Certis反光衣(軍) S,反光衣 Reflective Vest,1.0,3.0,2.0,4.0,3.0,2.0,2.0,2.0,1.0,7.0,0.0,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
552,黑色Certis Polo 恤衫 (軍) L,軍裝上衣 Military Top,0.0,0.0,0.0,6.0,0.0,0.0,4.0,2.0,0.0,0.0,5.0,0.0,1.0
553,黑色Certis Polo 恤衫 (軍) M,軍裝上衣 Military Top,3.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,1.0
554,黑色Certis Polo 恤衫 (軍) S,軍裝上衣 Military Top,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
555,黑色Certis Polo 恤衫 (軍) XL,軍裝上衣 Military Top,0.0,0.0,3.0,5.0,3.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,1.0


In [299]:
joined_data = pd.merge(monthly_sales, sales_forecasts, on=["Item_name"], how="left")
joined_data['Forecasted_Sales']=joined_data['Forecasted_Sales'].fillna(0)
joined_data['Category'] = monthly_sales['Category']
joined_data

,Item_name,Category,2022-08,2022-09,2022-10,2022-11,2022-12,2023-01,2023-02,2023-03,2023-04,2023-05,2023-06,2023-07,2023-08,Forecasted_Sales
0,Certis反光衣(軍) 2XL,反光衣 Reflective Vest,0.0,2.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,2.0,2.0
1,Certis反光衣(軍) 3XL,反光衣 Reflective Vest,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
2,Certis反光衣(軍) L,反光衣 Reflective Vest,5.0,4.0,2.0,4.0,5.0,5.0,1.0,1.0,3.0,10.0,4.0,3.0,7.0,5.0
3,Certis反光衣(軍) M,反光衣 Reflective Vest,5.0,7.0,2.0,1.0,6.0,3.0,2.0,2.0,3.0,7.0,4.0,1.0,3.0,3.0
4,Certis反光衣(軍) S,反光衣 Reflective Vest,1.0,3.0,2.0,4.0,3.0,2.0,2.0,2.0,1.0,7.0,0.0,2.0,2.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
552,黑色Certis Polo 恤衫 (軍) L,軍裝上衣 Military Top,0.0,0.0,0.0,6.0,0.0,0.0,4.0,2.0,0.0,0.0,5.0,0.0,1.0,1.0
553,黑色Certis Polo 恤衫 (軍) M,軍裝上衣 Military Top,3.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,1.0,0.0
554,黑色Certis Polo 恤衫 (軍) S,軍裝上衣 Military Top,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-1.0
555,黑色Certis Polo 恤衫 (軍) XL,軍裝上衣 Military Top,0.0,0.0,3.0,5.0,3.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,1.0,1.0


In [309]:
joined_data.to_csv('forecasted_Sales.csv',encoding='utf-8',index=False)

In [108]:
# pivoted_df = cleaned_df.pivot_table(index=["Staff No.", "Staff Name", "Gender", "Site", "Ranking", "Item Name"],
#                                     columns=["Transaction Date"],
#                                     values="Out", aggfunc='sum')

# pivoted_df.fillna(0, inplace =True)
# pivoted_df.head()

In [109]:
# reset_table = pivoted_df.reset_index()
# reset_table